In [1]:
!pip install openai
!pip install graphviz

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
    Uninstalling httpcore-0.9.1:
      Successfully uninstalled httpcore-0.9.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.13.3
    Uninstalling httpx-0.13.3:
      Successfully uninstalled httpx-0.13.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have httpx 0.28.1 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
import requests
import os
import time

# Load API Key securely
API_KEY = os.getenv("OPENROUTER_API_KEY") 

# OpenRouter API URL
URL = "https://openrouter.ai/api/v1/chat/completions" 

def intent_recognition(user_message):
    def call_api():
        messages = [
            {
                "role": "system",
                "content": """
You are an AI that classifies user messages into one of the following support categories.
Respond ONLY with the corresponding category number (e.g., 1, 2, 3, ..., 24). 
Do NOT include the category name, any explanations, or multiple categories.

Categories:
1. Need emotional support
2. Need immediate crisis support
3. Struggling with anxiety or panic attacks
4. Want therapy recommendations
5. Looking for professional help
6. Feeling depressed or hopeless
7. Seeking ways to cope with loneliness
8. Have questions about medication or psychiatry
9. Struggling with grief or loss
10. Need support for trauma or PTSD
11. Facing financial stress or money issues
12. Need help with parenting or family stress
13. Experiencing bullying or harassment
14. Seeking advice on domestic abuse or legal issues
15. Struggling with self-esteem or identity
16. Need motivation or productivity tips
17. Exploring spirituality or life purpose
18. Looking for mindfulness or relaxation techniques
19. Feeling stressed about work or studies
20. Need career or job loss advice
21. Experiencing workplace toxicity or discrimination
22. Struggling with social anxiety or communication
23. Need advice on relationships or social issues
24. Other concerns that don’t fit above
                """
            },
            {"role": "user", "content": "I just feel like no one understands me anymore."},
            {"role": "assistant", "content": "1"},
            {"role": "user", "content": "I’ve been thinking about ending everything."},
            {"role": "assistant", "content": "2"},
            {"role": "user", "content": "I can’t stop overthinking before social events. My heart races and I avoid them."},
            {"role": "assistant", "content": "3"},
            {"role": "user", "content": "What type of therapy is best for healing trauma?"},
            {"role": "assistant", "content": "4"},
            {"role": "user", "content": "Can you help me find a psychologist near me?"},
            {"role": "assistant", "content": "5"},
            {"role": "user", "content": "I’m tired all the time and feel like nothing matters."},
            {"role": "assistant", "content": "6"},
            {"role": "user", "content": "My friends have all moved away. I feel so disconnected."},
            {"role": "assistant", "content": "7"},
            {"role": "user", "content": "Are antidepressants safe long-term?"},
            {"role": "assistant", "content": "8"},
            {"role": "user", "content": "My dad passed away last month, and I can’t stop crying randomly."},
            {"role": "assistant", "content": "9"},
            {"role": "user", "content": "I’ve had nightmares about what happened years ago. It keeps coming back."},
            {"role": "assistant", "content": "10"},
            {"role": "user", "content": user_message}
        ]

        payload = {
            "model": "meta-llama/llama-2-13b-chat",
            "messages": messages
        }

        headers = {
            "Authorization": f"Bearer {API_KEY}",
            "Content-Type": "application/json",
            "HTTP-Referer": "yourwebsite.com",  # Optional
            "X-Title": "LLaMA Intent Classifier"
        }

        # Retry logic
        max_retries = 5
        for attempt in range(max_retries):
            try:
                response = requests.post(URL, json=payload, headers=headers)
                response.raise_for_status()
                result = response.json()
                if "choices" in result and result["choices"]:
                    content = result["choices"][0]["message"]["content"].strip()
                    return content
            except requests.exceptions.RequestException as e:
                print(f"⚠️ API request failed (Attempt {attempt+1}/{max_retries}): {e}")
                time.sleep(2 ** attempt)
        return None

    # Attempt API call
    intent_number = call_api()
    if intent_number:
        return intent_number

    print("❌ Failed to classify intent after multiple retries.")
    return "Error: Unable to determine intent."



In [23]:
sample_data = [
    ("I can't stop crying and I think about ending everything every day. It feels like there's no way out, and I just want the pain to stop.", "Need immediate crisis support"),
    ("How can I find a good therapist who understands depression? I feel like I need someone experienced with deep emotional struggles.", "Looking for professional help"),
    ("I just feel so alone lately and need someone to talk to. It’s like no one truly sees how much I’m hurting inside.", "Need emotional support"),
    ("What therapy works best for PTSD? I’ve been through a traumatic experience, and I need help figuring out how to move forward.", "Want therapy recommendations"),
    ("I keep having panic attacks and can't sleep at night. My heart races out of nowhere and I feel like I’m dying.", "Struggling with anxiety or panic attacks"),
    ("Everything feels meaningless and I'm constantly exhausted. Even simple tasks feel like climbing a mountain.", "Feeling depressed or hopeless"),
    ("How do I make new friends in a new city? I feel so disconnected and isolated, like I don’t belong anywhere.", "Seeking ways to cope with loneliness"),
    ("Are antidepressants safe? How do they work? I’m nervous about side effects but also feel like I need help.", "Have questions about medication or psychiatry"),
    ("I still cry every day after losing my mom last year. It’s been months, but the grief still hits me like a wave.", "Struggling with grief or loss"),
    ("I keep having nightmares about what happened in the accident. It plays over and over in my mind and I can’t seem to escape it.", "Need support for trauma or PTSD"),
    ("I lost my job and I don’t know how I’ll pay rent this month. I’m panicking every day and can’t think straight.", "Facing financial stress or money issues"),
    ("Raising my kids alone is so overwhelming right now. I feel like I’m constantly failing them and myself.", "Need help with parenting or family stress"),
    ("My coworkers keep making fun of my accent and it hurts. I feel humiliated and dread going to work every day.", "Experiencing bullying or harassment"),
    ("My partner has been hitting me and I don’t know what to do. I’m scared and feel completely trapped.", "Seeking advice on domestic abuse or legal issues"),
    ("I hate the way I look and don't feel good enough for anyone. I’m constantly comparing myself and never feel worthy.", "Struggling with self-esteem or identity"),
    ("I just can't find the motivation to do anything productive. Every day feels wasted and I hate myself for it.", "Need motivation or productivity tips"),
    ("I'm questioning what I believe and feel lost in life. I used to have purpose, but now I’m not sure why I’m even here.", "Exploring spirituality or life purpose"),
    ("How can I calm down when I’m stressed out? I need simple techniques because I feel overwhelmed almost every day.", "Looking for mindfulness or relaxation techniques"),
    ("I feel so burned out from school, I just want to quit. The pressure is too much, and I’m not sure I can keep going.", "Feeling stressed about work or studies"),
    ("I got laid off and I’m not sure what to do next. I feel like I’ve lost my identity and stability overnight.", "Need career or job loss advice"),
    ("My manager keeps giving me more work than others because I'm younger. I feel like I'm being taken advantage of and it's affecting my mental health.", "Experiencing workplace toxicity or discrimination"),
    ("Talking to people makes me so nervous I avoid it completely. I want to connect but I freeze up and don’t know what to say.", "Struggling with social anxiety or communication"),
    ("My relationship is falling apart and I don't know what to do. We argue constantly, and I’m afraid it’s beyond saving.", "Need advice on relationships or social issues"),
    ("I'm dealing with a lot but not sure where it fits in. Life feels chaotic and nothing makes sense anymore.", "Other concerns that don’t fit above"),
    ("The world just feels like it’s closing in on me lately. I don’t know how to breathe through the constant pressure.", "Feeling depressed or hopeless"),
    ("Every time I open up, I feel like I’m being a burden. I just want someone to actually care and listen.", "Need emotional support"),
    ("Are there natural ways to deal with depression besides medication? I’m looking for more holistic options.", "Want therapy recommendations"),
    ("I keep shaking and sweating during meetings. It’s ruining my confidence and making work unbearable.", "Struggling with anxiety or panic attacks"),
    ("It’s been over a year since the breakup, but I still cry myself to sleep thinking about them.", "Struggling with grief or loss"),
    ("I feel like my trauma defines me. I can't move past what happened, and it’s holding me back from living.", "Need support for trauma or PTSD"),
    ("The bills are piling up and I have no idea how to make ends meet. I feel like I’m drowning.", "Facing financial stress or money issues"),
    ("I’m a single parent working two jobs and barely surviving. No one really understands how hard this is.", "Need help with parenting or family stress"),
    ("People laugh at my gender expression and it’s crushing. I just want to be accepted for who I am.", "Experiencing bullying or harassment"),
    ("I don’t feel safe in my own home anymore. My partner is emotionally abusive and I feel stuck.", "Seeking advice on domestic abuse or legal issues"),
    ("I don’t know who I am anymore. My confidence is gone, and I hate the person I’ve become.", "Struggling with self-esteem or identity"),
    ("I keep procrastinating and can’t seem to focus. My to-do list keeps growing and I feel paralyzed.", "Need motivation or productivity tips"),
    ("What’s the meaning of all this? I wake up every day feeling disconnected from the world around me.", "Exploring spirituality or life purpose"),
    ("Can you teach me a few breathing exercises to reduce stress? I’ve heard mindfulness helps but I don’t know where to start.", "Looking for mindfulness or relaxation techniques"),
    ("Work is so overwhelming. I’m staying up late, missing meals, and still not getting everything done.", "Feeling stressed about work or studies"),
    ("I can’t find a new job and I’m starting to feel hopeless. What if I never recover from this setback?", "Need career or job loss advice"),
    ("My team leaves me out of decisions and jokes behind my back. I feel excluded and devalued.", "Experiencing workplace toxicity or discrimination"),
    ("Even when I try to hang out with people, I get anxious and feel like I don’t belong. It’s easier to stay home.", "Struggling with social anxiety or communication"),
    ("I love my partner, but we keep fighting and I don’t know how to fix it. I’m afraid we’re drifting apart.", "Need advice on relationships or social issues"),
    ("I feel so overwhelmed by everything—family, finances, my health. I don’t know where to begin.", "Other concerns that don’t fit above"),
    ("Sometimes I imagine what it would be like if I disappeared. I don’t think anyone would even notice.", "Need immediate crisis support"),
    ("Is it normal to be afraid of opening up to a therapist? I want help, but I feel too vulnerable.", "Looking for professional help"),
    ("I miss my dad so much it physically hurts. Every little thing reminds me of him, and I break down.", "Struggling with grief or loss"),
    ("No matter how hard I try, I can’t seem to let go of what happened in the past. The trauma feels fresh every day.", "Need support for trauma or PTSD"),
    ("I haven’t been able to sleep or eat properly since losing my job. I feel like a failure.", "Facing financial stress or money issues")
]


In [ ]:
y_true = []
y_pred = []

for text, true_label in sample_data:
    prediction = intent_recognition(text)
    y_true.append(true_label)
    y_pred.append(prediction)
    print(f"Input: {text[:50]}... \nPredicted: {prediction} | Actual: {true_label}\n")

report = classification_report(y_true, y_pred, labels=INTENT_CATEGORIES, zero_division=0)
print(report)


❌ Failed to classify intent after multiple retries.
Input: I can't stop crying and I think about ending every... 
Predicted: Error: Unable to determine intent. | Actual: Need immediate crisis support

Input: How can I find a good therapist who understands de... 
Predicted: 11 | Actual: Looking for professional help

Input: I just feel so alone lately and need someone to ta... 
Predicted: Please remember that these category titles are just examples, and the actual content of each message may vary. | Actual: Need emotional support

Input: What therapy works best for PTSD? I’ve been throug... 
Predicted: In order to provide additional information and better support, do you have a specific question in mind related to these categories that you would like me to direct you to appropriate resources for (e.g. "Could you recommend me a therapist near my location for X type of issue?" or "What are the best self-care techniques to manage Y problem?"). please let me know and I am here to Help. | Ac